


# Reading Data




In [ ]:
RatingsPath = r'C:\Users\ASUS\Desktop\CF\data\ml-1m\ratings.dat'
Open = open(RatingsPath, 'r')
Rating = Open.read()

raw = Rating.split("\n")

In [ ]:
DATA = [[],[],[]]
mat = []
check = 1
t = []
for i in range(0, len(raw)-1):
  temp = raw[i].split('::')
  userID = int(temp[0])
  itemID = int(temp[1])
  val = int(temp[2])
  ls =[]
  # ls.append(userID-1)
  ls.append(itemID-1)
  ls.append(val)
  if(userID==check):
    t.append(ls)
  else:
    mat.append(t)
    check = userID
    t = []
  DATA[0].append(userID)
  DATA[1].append(itemID)
  DATA[2].append(val)

In [ ]:
MatrixForTrain = []
MatrixForTest = []

def build(mat):
  items = 3952
  users = 6040
  for i in range(users):
    temp = []
    for j in range(items):
      temp.append(-1);
    mat.append(temp)
    # MatrixForTest.append(temp)
    # print(len(MatrixForTrain))

build(MatrixForTrain)
build(MatrixForTest)

In [ ]:
def function(user, train, idx, test):
    num = int(len(user)*0.7)
    for i in range(0, num):
        ls = user[i]
        train[idx][ls[0]] = ls[1]
    
    for i in range(i, len(user)):
        ls = user[i]
        test[idx][ls[0]] = ls[1]


In [ ]:
for i in range(0, len(mat)):
    function(mat[i], MatrixForTrain, i, MatrixForTest)

In [ ]:
def MAE(X, Y):
  count = 0
  for i in range(len(X)):
    count = count + (abs(X[i] - Y[i]))
  return float(count)/float(len(X))

  def NMAE(X, Y):
  val = MAE(X, Y)
  i = max(X)-min(X)
  if(i!=0):
    return val/i
  return val

# Nueral Network to obtain the weights

In [ ]:
import numpy as np
import pandas as pd
from tensorflow import keras
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Dropout
from keras.models import Model

In [ ]:
dictionary = {
    'userId' : DATA[0],
    'movieId' : DATA[1],
    'rating' : DATA[2]
}

data = pd.DataFrame(dictionary)

In [ ]:
num_users = len(data['userId'].unique())
num_items = len(data['movieId'].unique())

spliting dataSet

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2)

defining model

In [ ]:
def ncf_model(num_users, num_items, embedding_size=32, hidden_units=[64, 32, 16]):
    
    user_input = Input(shape=(1,), name='user_input')
    item_input = Input(shape=(1,), name='item_input')
    
    
    user_embedding = Dense(embedding_size, name='user_embedding')(user_input)
    item_embedding = Dense(embedding_size, name='item_embedding')(item_input)
    
    
    concatenated = keras.layers.Concatenate()([user_embedding, item_embedding])
    
   
    for i, units in enumerate(hidden_units):
        concatenated = Dense(units, activation='relu', name=f'hidden_layer_{i}')(concatenated)
        concatenated = Dropout(0.5)(concatenated)
    
  
    output = Dense(1, activation='sigmoid', name='output_layer')(concatenated)
    
    
    model = Model(inputs=[user_input, item_input], outputs=output)
    
    return model

compiling model

In [ ]:
model = ncf_model(num_users, num_items)
model.compile(loss='binary_crossentropy', optimizer='adam')

Training the model

In [ ]:
history = model.fit([train_data['userId'], train_data['movieId']], train_data['rating'], batch_size=64, epochs=10, validation_split=0.2, verbose = 0)

Making prediction

In [ ]:
predictions = model.predict([test_data['userId'], test_data['movieId']])

6252/6252 [==============================] - 9s 1ms/step


## getting Output

In [ ]:
output_from_neural_network = []
for i in range(0, len(predictions)):
    output_from_neural_network.append(float(predictions[i][0]))

In [ ]:
def update_output(output_from_neural_network):
    for i in range(len(output_from_neural_network),306922):
        output_from_neural_network.append(1)

update_output(output_from_neural_network)


#Reading preproceesing data generated by the knowledge Graph

In [ ]:
preprocessDataPath = r'C:\Users\ASUS\Desktop\CF\preprocess.txt'
Open1 = open(preprocessDataPath, 'r')
preprocessRaw = Open1.read()
preProcess = preprocessRaw.split("\n")

## Reading preprocessing data

In [ ]:
def getRowForPreProcess(preProcess, idx):
    row = preProcess[idx].strip(']')
    row = row.strip('[')
    row = row.split(',')
    for j in range(len(row)):
        row[j] = float(row[j])
    
    return row

def getPreProcessMatrix(preProcess):
    matrix = []
    for idx in range(0, len(preProcess)-1):
        # print(idx)
        matrix.append(getRowForPreProcess(preProcess, idx))
    matrix.append([])
    return matrix

In [ ]:
PrepocessData = getPreProcessMatrix(preProcess)

In [ ]:
userMean =[]
for i in range(len(MatrixForTrain)):
  tempCount = 0
  temp = 0
  for j in range(len(MatrixForTrain[i])):
    if(MatrixForTrain[i][j]!=-1):
      temp = temp + MatrixForTrain[i][j]
      tempCount = tempCount + 1
  if(tempCount==0):
    userMean.append(1)
  else:
    userMean.append(float(temp)/tempCount)

#Helper function

In [ ]:
def helper(matTrain, matSimilarity, userMean, item, user, K):
  similarities = {}
  for i in range(0, 943):
    if(i!=user):
      mi = min(user, i)
      ma = max(user, i)
      sim = matSimilarity[mi][ma-mi-1]
      if(mi==user):
        similarities[ma] = sim
      else:
        similarities[mi] = sim
  sortedSimilarity = sorted(similarities.items(), key=lambda x:x[1])
  simItem = []
  i = len(sortedSimilarity)-1
  while(len(simItem)!=K and i>=0):
    u = sortedSimilarity[i][0]
    it = matTrain[u][item]
    if(it!=-1):
      ls = []
      ls.append(sortedSimilarity[i][1])
      ls.append(it- userMean[u])
      simItem.append(ls)
    i = i - 1

  upper = 0
  
  lower = 0
  
  for i in range(len(simItem)):
    lower = lower + abs(simItem[i][0])
    upper = upper + simItem[i][0]*simItem[i][1]

  if(lower==0):
    return round(userMean[user] + 0)
  ans = upper/lower
  ans = round(userMean[user] + ans)
  return ans

In [ ]:
def getRootMeanSquareValue(X, Y):
    val = 0
    for i in range(len(X)):
        num = abs(X[i]-Y[i])
        num = num**2
        val = val + num
    
    val = val/len(X)
    
    return val**0.5


In [ ]:
def all(listForTest, matrixUser, matrixSimilarity, userMean, K):
  ls = []
  for i in range(len(listForTest)):
    u = listForTest[i][0]
    i = listForTest[i][1]
    temp = helper(matrixUser, matrixSimilarity,userMean, i, u, K)
    ls.append(temp)
  return ls

In [ ]:
def hm(matforTest, listForTest):
  hm = []
  for i in range(len(listForTest)):
    temp = matforTest[listForTest[i][0]][listForTest[i][1]]
    hm.append(temp)
  return hm

In [ ]:
def getUserItemForTest(mat):
  ls = []
  for i in range(len(mat)):
    for j in range(len(mat[i])):
      if(mat[i][j]!=-1):
        temp = []
        temp.append(i)
        temp.append(j)
        ls.append(temp)
  return ls

In [ ]:
listForTest = getUserItemForTest(MatrixForTest)

# combining knowledge graph data to the svd to get final output

In [ ]:
result = all(listForTest, MatrixForTest, PrepocessData, userMean, 5)  

In [ ]:
test = hm(MatrixForTest, listForTest)

In [ ]:
def updateFin(matForUser, listOfIndex, values, userMean):
  for i in range(len(listOfIndex)):
    matForUser[listOfIndex[i][0]][listOfIndex[i][1]] = values[i]
  
  for i in range(len(matForUser)):
    for j in range(len(matForUser[i])):
      if(matForUser[i][j]==-1):
        matForUser[i][j] = userMean[i]
      matForUser[i][j] = float(matForUser[i][j])  

In [ ]:
updateFin(MatrixForTrain, listForTest, result, userMean)

In [ ]:
def fillMean(matforUser, userMean):
  for i in range(len(matforUser)):
    for j in range(len(matforUser[i])):
      if(matforUser[i][j]==-1):
        matforUser[i][j] = userMean[i]
      matforUser[i][j] = float(matforUser[i][j])

In [ ]:
fillMean(MatrixForTrain, userMean)

In [ ]:
def getResullts(listOfIndex, R_hat):
  resultForUser= []
  for i in range(len(listOfIndex)):
    resultForUser.append(float(round(R_hat[listOfIndex[i][0]][listOfIndex[i][1]])))
  return resultForUser

#Combining the output of Neural network and svd together to get final output

In [ ]:
import numpy as np
from scipy.sparse.linalg import svds
def final_output(output_from_neural_network,output_from_mf):
  lst=[]
  for i in range(len(output_from_neural_network)):
    num  = output_from_neural_network[i]*output_from_mf[i]
    lst.append(num)

  return lst  


def finalPredictedResults(output_from_neural_network,matForTrain, listForTest, testU, K, userMean):
  R = [[]]
  for i in range (100):
    U, sigma, v = svds(np.array(matForTrain), K)
    sigma = np.diag(sigma)
    R = np.dot(np.dot(U, sigma), v)
  for i in range(len(R)):
    for j in range(len(R[0])):
      R[i][j] = int(round(R[i][j]))
  resultOfUser = getResullts(listForTest, R)

  # combining neural network output to the svd
  final_predicted_output=final_output(output_from_neural_network,resultOfUser)
  print("RMSE: ", getRootMeanSquaresValue(testU, final_predicted_output))
 

# RMSE value

In [ ]:
finalPredictedResults(output_from_neural_network, MatrixForTrain, listForTest,  test, 25, userMean)

RMSE:  0.8916308860328659
